# Electronic Spectroscopy with VOTCA and LAMMPS

In this tutorial we will investigate the solvatochromic shift in the spectrum of acetone in water using LAMMPS and VOTCA.

## Before we start

VOTCA has a command line interface, but no python interface (yet!). We don't use jupyter for its python capabilities in this tutorial, but for its ability to show and run commands with comments in Markdown. Some things you need to know

* The exclamation mark (`!`) runs the cell as a Linux/Unix command in the terminal, hence all lines starting with `!` are not python, but shell commands and can also be run in the terminal (or docker container) directly.
* VOTCA requires certain environment variables to be set. This is done with the `VOTCARC.bash` or `VOTCARC.csh` script in the `bin` directory of VOTCA. It needs to be sourced before the jupyter notebook server is started or any VOTCA related commands are executed in the terminal. As an example: suppose VOTCA is installed in `/home/user/Software/votca`, then to set all environment variables and start jupyter run
```bash
source /home/user/Software/votca/bin/VOTCARC.bash
jupyter notebook
```
In the docker container you only need to run
```bash
source VOTCARC.bash
```

### Testing the environment
To check if the environment variables are set correctly, we will execute the `xtp_tools` command with the help flag `-h`. The output should be the VOTCA banner and a list of allowed options. If you get something else make sure you sourced the `VOTCARC` script.

In [4]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:05)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          Tool user options.
  -t [ --nthreads ] arg (=1)    number of threads to create
  -e [ --execute ] arg        Name of Tool to run
  -l [ --list ]               Lists all available Tools
  -d [ --description ] arg    Short description of a Tools
  -c [ --cmdoptions ] arg     Modify options via command line by e.g. '-c 
                         

## Creating the topology and parametrizing the model

As a starting point for the QM/MM calculations we need a topology (i.e. the layout of the system). This topology is not just a snap shot of an MD trajectory, eventhough it is a major part of it. Besides the MD trajectory, we need information about

1. The ground state geometry of the molecules,
2. The multipole representation of the molecules and
3. Information about the polarizabililty of the molecules.

This information can be obtained in different ways. For this tutorial we have chosen to use the ORCA dft package, for which VOTCA has an interface.  The necessary files and the ORCA input files that generated them can be found in the `DFT_ORCA` folder. 

### 1. Getting the ground state geometry of a single molecule

To compute the ground state geometry, a DFT optimization calculation needs to be performed. In this tutorial we have done this with ORCA. The input files and optimized geometry can be found in the `DFT_ORCA` folder.

### 2. The multipole representation: converting the ORCA files to VOTCA input files

To get the multipole representation of the molecules we use partial charges computed with the CHELPG scheme of ORCA. We need to convert the ORCA CHELPG output to VOTCA multipole files (.mps). This is done with the following command. 


In [5]:
!xtp_tools -e log2mps -o OPTIONS/log2mps_water.xml

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:05)

Initializing tool
... log2mps 
... ... DFT_ORCA/water/chelpg.log => MP_FILES/water_n.mps
Evaluating tool
... log2mps  Using 1 threads
... ... Using package <orca>
... ... Parsing DFT_ORCA/water/chelpg.log
... ... Getting the coordinates
... ... Getting charges
... ... Getting charges
... ... 3 QM atoms, total charge Q = 0

The previous command shows the general structure of running a program, which is always the same

```bash
xtp_<executableType> -e <whichCalculationToRun> -o <optionsFile> ...
```

The `executableType` indicates what type of executable is used. VOTCA has different executable types based on the kind of calculation, is the calculaton based on an MD topology, does it need parallelization support etc. We will see the different types shortly.

`whichCalculationToRun` indicates what should be done, a DFT-GWBSE calculation, a file conversion etc. Finally the `optionsFile` is the file with all the input options. 

Lets look at the `log2mps_water.xml` option file

```xml
<options>
	<log2mps>
		<dftpackage>orca</dftpackage>
		<input>DFT_ORCA/water/chelpg.log</input>
		<output>MP_FILES/water_n.mps</output>
	</log2mps>
</options>
```

We see that there are three options to set. The input file (which is the orca output log in this case), the name of the output file and which DFT package was used to do the calculation.

### How to find out which options are available?

There are two ways to find out which options are available for a tool. Every tool has its own description, you can get it for the `log2mps` tool by running the following command (where `-d` means describe).

In [6]:
!xtp_tools -d log2mps

 log2mps:   Generates an mps-file (with polar-site definitions) from a QM log-file
   OPTION      DEFAULT    UNIT           DESCRIPTION
   job_name    (system)                  Input file name without extension, also used for intermediate files
   dftpackage  (orca)                    QM package name
   input       (OPTIONAL)                dftpackage log file to read from, otherwise use job_name
   output      (OPTIONAL)                MPs file with charges, if not given use job_name
Done - stopping here


Alternatively you can generate a starting xml file (with help messages) using `xtp_tools -p <tool_name> -o <output>.xml`, which you can manually edit to fit your needs. This file will contain all available options, mandatory and optional. For `log2mps` it can be generated and viewed with the following commands. Note that the result is much better when the option file is viewed in an editor with syntax highlighting (VS code for example).

In [7]:
!xtp_tools -p log2mps -o temp_log2mpsOptions.xml
!echo "**************"
!cat temp_log2mpsOptions.xml

Writing options for calculator log2mps to temp_log2mpsOptions.xml
Done - stopping here
**************
<options>
<log2mps help="Generates an mps-file (with polar-site definitions) from a QM log-file" label="tool:log2mps">
	<job_name default="system" help="Input file name without extension, also used for intermediate files">system</job_name>
	<dftpackage choices="orca" default="orca" help="QM package name">orca</dftpackage>
	<input default="OPTIONAL" help="dftpackage log file to read from, otherwise use job_name"/>
	<output default="OPTIONAL" help="MPs file with charges, if not given use job_name"/>
</log2mps>
</options>



### The result of `log2mps`

The result of `log2mps` is an mps file, for water it will look like

```text
! GENERATED BY VOTCA::XTP::::LOG2MPS (log-file='../DFT_ORCA/water/chelpg.log' : 3 QM atoms)
! N=3 Q[e]=+0.0000000
Units angstrom
  O +0.0000000 +0.0000000 -0.0043320 Rank 0
    -0.7585550
     P +0.8370000
  H +0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3787500
     P +0.4960000
  H -0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3798050
     P +0.4960000
```

We see the atom positions, followed by their partial charge (rank 0, for higher ranks dipoles and quadrupoles are also specified) and the line with the `P` indicating the polarizability. But something is off with the polarizability it is just a single number. 

### 3. Fitting Polarizabilities
To obtain the "real" polarizability we will fit the atomic polarizabilities, such that they represent the total molecular polarizability (computed with ORCA in this case) as close as possible. This is done with the `molpol` tool. Lets run the tool.

In [8]:
!xtp_tools -e molpol -o OPTIONS/molpol_water.xml

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:05)

Initializing tool
... molpol 
 ... Using package <orca>
 ... Getting polarizabilityEvaluating tool
... molpol  Using 1 threads
Iteration 1 of 100 target:6.46783 current:9.41669

Iteration 2 of 100 target:6.46783 current:6.89261

Iteration 3 of 100 target:6.46783 current:6.54134

Iteration 4 of 100 target:6.46783 current:6.48093

Iteration 5 of 100 target:6.46783 current:6.47018

Iteration 6 of 100 target:6.46783 current:6.46825

... ... Iterative refinement : *CONVERGED*
... ... Scaling coefficient  : -6.50073e-05
First principle polarization [A^3]
 1.18054        0        0
       0 0.772045       -0
       0       -0 0.965972
Scaled classical polar

The options for this tool are the original mps file, the name of the new mps file, which package is used for the polarization calculation and where to find the logfile with the molecular polarizability information. The option file:
```xml
<options>
	<molpol>
		<input>MP_FILES/water_n.mps</input>
		<output>MP_FILES/water_n_pol.mps</output>
		<mode>qmpackage</mode>
		<qmpackage>orca</qmpackage>
		<logfile>DFT_ORCA/water/chelpg.log</logfile>
	</molpol>
</options>
```

If you check the output file now, you will see that it added atomic polarizabilities.

In [9]:
!cat MP_FILES/water_n_pol.mps

! GENERATED BY VOTCA::XTP::MOLPOL (OPTIMIZED)
! N=3 Q[e]=+0.0000000
Units angstrom
  O +0.0000000 +0.0000000 -0.0043320 Rank 0
    -0.7585550
     P +0.5320935 +0.0000000 +0.0000000 +0.5320935 +0.0000000 +0.5320935
  H +0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3787500
     P +0.3153146 +0.0000000 +0.0000000 +0.3153146 +0.0000000 +0.3153146
  H -0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3798050
     P +0.3153146 +0.0000000 +0.0000000 +0.3153146 +0.0000000 +0.3153146


### Repeating the process for other molecules

This process needs to be repeated for all molecules in the system. Generating the files for acetone for example can be done by running the following

In [10]:
!xtp_tools -e log2mps -o OPTIONS/log2mps_acetone.xml
!echo " " # we only need it to clearly separate the two commands
!xtp_tools -e molpol -o OPTIONS/molpol_acetone.xml

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:05)

Initializing tool
... log2mps 
... ... DFT_ORCA/acetone/chelpg.log => MP_FILES/acetone_n.mps
Evaluating tool
... log2mps  Using 1 threads
... ... Using package <orca>
... ... Parsing DFT_ORCA/acetone/chelpg.log
... ... Getting the coordinates
... ... Getting charges
... ... Getting charges
... ... 10 QM atoms, total charge Q = 1e-06 
========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 

## Putting everything together (mapping)

We now have all the input files we need. To combine them with the snap shot of the MD trajectory we need to perform a mapping. For this mapping we need to create an input file, the *mapping file*. This file tells VOTCA which geometry to use for different types of calculations. As an example, for a QM calculation on acetone we can't use the distorted MD geometry, because it won't be close to the ground state geometry of the molecule and hence give a weird energy value. Hence we need to replace the MD geometry by the QM groundstate geometry. To use the QM geometry for the QM calculation we map the QM geometry of the molecule on the MD geometry.

The mapping file itself looks quite complicated so we start by looking only at a small part that will give us the main idea.

Suppose we want to map the ground state geometry of water onto the MD trajectory, we would at lease need the following input

```xml
<mdatoms>2:O76:10 2:H77:11 2:H77:12</mdatoms>
<qmatoms>0:O 1:H 2:H</qmatoms>
<localframe>0 1 2</localframe>
```

where `mdatoms` is the order of the MD geometry in the MD trajectory (.traj from LAMMPS) and `qmatoms` gives the order in which the QM geometry (.xyz file from ORCA/VOTCA or other package) should be mapped to the MD geometry. Hence the first MD atom is mapped to the first QM atom etc. The colon `:` notation is used to provide a bit more info. In the `qmatoms` tags we have `atomID:atomType` in the `mdatoms` tag we have `resID:atomType:atomID`. The `localframe` is then used to specify the atoms on which we base the mapping (i.e. the positions that are used to compute the rotation matrix and translation between the MD and QM geometries).


What follows is a more complete mapping file. 

```xml
<molecule>
    <mdname>C3H6O1</mdname>
    <segments>
        <segment>
            <name>ACETONE</name>
            <qmcoords_n>DFT_ORCA/acetone/acetoneOpt.xyz</qmcoords_n>
            <qmcoords_e>DFT_ORCA/acetone/acetoneOpt.xyz</qmcoords_e>
            <qmcoords_h>DFT_ORCA/acetone/acetoneOpt.xyz</qmcoords_h>
            <multipoles_n>MP_FILES/acetone_n_pol.mps</multipoles_n>
            <map2md>0</map2md>
            <fragments>
                <fragment>
                <name>acetone</name>
                <mdatoms>1:O223:0 1:C222:1 1:C80:2 1:C80:3 1:H85:4 1:H85:5 1:H85:6 1:H85:7 1:H85:8
                , → 1:H85:9</mdatoms>
                <qmatoms>0:O 1:C 2:C 3:C 4:H 5:H 6:H 7:H 8:H 9:H</qmatoms>
                <mpoles>0:O 1:C 2:C 3:C 4:H 5:H 6:H 7:H 8:H 9:H</mpoles>
                <weights>16 12 12 12 1 1 1 1 1 1</weights>
                <localframe>0 1 3</localframe>
                </fragment>
            </fragments>
        </segment>
    </segments>
</molecule>
```

Under the tags `qmcoords_x` and `multipoles_x` we can specify the different files with the geometries for the multipole expansion and qm calculations. The `x` can be any of `n,e,h` which is mostly relevant for charge transport where we might also want to use different geometries for hole (cation, h), electron (anion, e) and ground states (n).

In [11]:
!xtp_map -t LAMMPS/system.data -c LAMMPS/traj1.dump -s OPTIONS/mapping.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_map, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:05)


angle
atom
bond
full
molecule

These styles use the following formats in the atom block:
atom-ID molecule-ID atom-type charge x y z
atom-ID molecule-ID atom-type charge x y z nx ny nz
atom-ID molecule-ID atom-type x y z
atom-ID molecule-ID atom-type x y z nx ny nz
atom-ID atom-type x y z
atom-ID atom-type x y z nx ny nz

Unable to associate mass 15.035 with element assuming pseudo atom, assigning name Bead6 .
Unable to associate mass 15.035 with element assuming pseudo atom, assigning name Bead9 .
Unable to associate mass 15.035 with element assuming pseudo atom, assigning name Bead10 .
Unable to associate mass 15.035 with element assuming pseudo atom

This command will create a VOTCA state file based on the LAMMPS trajectory and system data and the mapping file. The state file is standard HDF5 and can be inspected with an HDF5 viewer like `Silx`. To check what the mapping procedure actually did we can generate QM and MD pdb files that we can compare in VMD. To generate the pdb file

In [13]:
!xtp_run -e mapchecker -o OPTIONS/mapchecker.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:24:35)
votca_csg, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:31)
votca_tools, version 2022-dev gitid: bb12c18 (compiled Aug 10 2021, 12:20:05)

Initializing calculator
... mapchecker
1 frames in statefile, Ids are: 798500 
Starting at frame 798500
Evaluating frame 798500
Import MD Topology (i.e. frame 798500) from state.hdf5
.... 
... mapchecker
 Using 1 threads
Writing segments to md_segments_step_798500.pdb
Writing qmmolecules to qm_segments_n_step_798500.pdb
Changes have not been written to state file.


To visualize the result start VMD and load the `md_segments_step_798500.pdb` and `qm_segments_step_798500.pdb` files as separate molecules. You will see for every molecule the MD geometry and mapped onto it the QM geometry. If there are any large differences, you might need to pick a better `localframe`.

# Setup the QM/MM jobfile and run the calculation

The next step is to specify for which molecules and we want to perform the QM/MM calculation and which molecules are part of the QM and MM region. What follows is part of the `qmmm.xml` option file:

```xml
<qmmm>
  <io_jobfile>
    <states>n s1</states>
    <segments>0</segments>
    <use_gs_for_ex>true</use_gs_for_ex>
  </io_jobfile>
  <regions>
    <qmregion>
      <id>0</id>
      <state>jobfile</state>
      <segments>0:n</segments>
      .
      .
    </qmregion>
    <polarregion>
      <id>1</id>
      <cutoff>
        <geometry>n</geometry>
        <radius>0.9</radius>
      </cutoff>
    </polarregion>
  </regions>
</qmmm>
```

Within the `io_jobfile` tag we find the information about which atoms to include in our calculation. In this case we want a groundstate `n` and first singlet level `s1` calculation on the first segment `0` (the acetone molecule) and we use the ground state density for excited state calculations (it is possible to provide a different geometry). Next in the `regions` section we find how the QM and MM region are specified. Region `0` is a qm region, the state of the calculation is based on the jobfile, hence it will be `n` since that is what we provided in the `io_jobfile` tag. The second region `1` is a polar region and is based on a cutoff in this case with a radius of 0.9nm. Which means that all water molecules in a ball with radius 0.9nm around the zeroth segment (the acetone molecule) are included in the calculation.

Now that we have the input file, we can generate the jobfile and run the calculations.

In [ ]:
!xtp_parallel -e qmmm -o OPTIONS/qmmm.xml -f state.hdf5 -j "write"

In [ ]:
!xtp_parallel -e qmmm -o OPTIONS/qmmm.xml -f state.hdf5 -j "run" -x 4

# Results

The results are stored in the `qmmm_jobs.xml` file. What follows is an example part

```xml
<output>
  <regions>
    <region Tot_charge="-3.200000e+01" id="0" size="1" type="qmregion">
      <E_total>-5252.342575</E_total>
    </region>
    <region Tot_charge="0.000000e+00" id="1" size="101" type="polarregion">
      <E_static>-30.978427</E_static>
      <E_polar>-9.246523</E_polar>
      <E_total>-40.224950</E_total>
    </region>
  </regions>
  <E_tot>-5292.567526</E_tot>
  <Compute_Time>24</Compute_Time>
  <Total_Charge>-32.000000</Total_Charge>
  <Iterations>4</Iterations>
</output>
```

We see here the different energies of the regions and the total energy. Note that this is just a single calculation. These QM/MM calculation only make sense when one look at differences. To get the correct singlet energy for example one needs to subtract the ground state energy from the singlet energy, to obtain the correct number. 